# Variable Processing

First, load in the full data set from S3.

In [3]:
df = spark.read.parquet('dbfs:/mnt/bigdataproject649/parquet5/')

How many observations are there in your data set?

In [5]:
# df.count()

Which variables should we focus on?

In [7]:
df.printSchema()

Get a small selection of columns including birth_year.

In [9]:
import pyspark.sql.functions as func
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
from datetime import datetime
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import min as _min
from pyspark.sql.functions import max as _max

# udf
#datediff, to_date, lit

status

In [11]:
# df_status = df.select(func.col("customer_info.line_id").alias("line_id"),
#                       func.col("status"))
                     
# df_status.dropDuplicates()
# df_status.count()
# display(df_status)

customer_info

In [13]:
df_customer_info = df.select(func.col("customer_info.line_id").alias("line_id"),
                                    func.col("customer_info.first_activation_date").alias("first_activation_date"),
                                    func.col("customer_info.lifetime_redemptions").alias("lifetime_redemptions"),
                                    func.col("customer_info.lifetime_revenues").alias("lifetime_revenues"))
#                                    (func.col("customer_info.lifetime_redemptions")/func.col("customer_info.lifetime_revenues")).alias("remem_over_revenue"))

In [14]:
# display(df_customer_info.select('lifetime_redemptions').orderBy(df_customer_info['lifetime_redemptions']))

In [15]:
# df_customer_info = df_customer_info.withColumn("first_activation_period",func.datediff(func.to_date(func.lit("2020-03-19")),func.to_date("first_activation_date")))

In [16]:
df_customer_info.count()

In [17]:
display(df_customer_info)

active&inactive tenure

In [19]:
df.select(func.col('status')).groupby('status').count().show()

In [20]:
df_status = df.select(func.col("customer_info.line_id").alias("line_id"),
                      func.col("status"))

In [21]:
# df_status.count()

In [22]:
# active_join
df_status_active = df_status.filter(df_status['status']=='Active')
# df_status_active.count()
active_first_date = df.select(func.col("customer_info.line_id").alias("line_id"),
                              func.col("customer_info.first_activation_date").alias("first_date"))
df_active_join = df_status_active.join(active_first_date,df_status_active.line_id == active_first_date.line_id,how='left').\
drop(active_first_date.line_id)
df_active_join.count()

In [23]:
active_first_date.select("first_date").filter(active_first_date['first_date'].isNull()).count()

In [24]:
df_active_join = df_active_join.withColumn("tenure_active",func.datediff(func.to_date(func.lit("2020-03-19")),func.to_date("first_date")))
df_active_join_id = df_active_join.groupby("line_id").agg(_max("tenure_active").alias("tenure"))

In [25]:
df_active_join_id.count()

In [26]:
df_status_inactive = df_status.filter(df_status['status']=='Inactive')
df_status_inactive.count()
inactive_date = df.select(func.col("customer_info.line_id").alias("line_id"),
                              func.col("customer_info.first_activation_date").alias("first_date"),
                               func.explode("deactivations.deactivation_date").alias("deactivation_date"))

In [27]:
# inactive_date = inactive_date.select('line_id','first_date','deactivation_date').filter(inactive_date['deactivation_date'].isNull()).drop()

In [28]:
# inactive_date_.select('line_id','first_date','deactivation_date').filter(inactive_date['deactivation_date'].isNull()).count()

In [29]:
df_inactive_join = df_status_inactive.join(inactive_date,df_status_inactive.line_id == inactive_date.line_id,how='left').drop(inactive_date.line_id)
# df_inactive_join.count()

In [30]:
df_inactive_join.select('line_id').distinct().count()

In [31]:
df_inactive_join = df_inactive_join.withColumn("tenure_inactive",func.datediff(func.to_date(func.to_date("deactivation_date")),func.to_date("first_date")))
df_inactive_join_id = df_inactive_join.groupby("line_id").agg(_max("tenure_inactive").alias("tenure"))

In [32]:
df_inactive_join_id.count()

In [33]:
# check missing values in deactivation
# df_inactive_join.select("deactivation_date").filter(df_inactive_join['deactivation_date'].isNull()).count()
# print(df_inactive_join.select("first_date").filter(df_inactive_join['first_date'].isNull()).count())
# print(df_inactive_join_id.select("tenure").filter(df_inactive_join_id['tenure'].isNull()).count())

In [34]:
# display(df_inactive_join.select("deactivation_date","first_date").filter(df_inactive_join['deactivation_date'].isNull()))

In [35]:
# net_join_concat = df_net_usage_table.join(df_concat,df_net_usage_table.line_id == df_concat.line_id,how='right').drop(df_concat.line_id)

In [36]:
df_concat = df_active_join_id.union(df_inactive_join_id)

In [37]:
df_concat.distinct().count()

In [38]:
display(df_concat)

network_usage_domestic

In [40]:
df_net_usage = df.select(func.col("customer_info.line_id").alias("line_id"),
                               func.explode("network-usage-domestic").alias("net_usage_by_day"))

In [41]:
# df_net_usage.select('line_id').distinct().count()

In [42]:
df_net_usage_table = df_net_usage.select("line_id", "net_usage_by_day.*")

In [43]:
display(df_net_usage_table)

In [44]:
#func_date =  func.udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
#df_net_usage_table = df_net_usage_table.withColumn('date', func_date(func.col('date')))
df_net_usage_table = df_net_usage_table.withColumn("sms", func.col("sms_in") + func.col("sms_out"))
df_net_usage_table = df_net_usage_table.withColumn("voice_duration", func.col("voice_duration_in") + func.col("voice_duration_out"))

In [45]:
df_net_usage_table=df_net_usage_table.groupBy("line_id").agg(_sum("sms").alias("sms_id"),_sum("voice_duration").alias("voice_duration_id"),_sum("voice_num_total").alias("voice_num_total_id"))

In [46]:
net_join_concat = df_concat.join(df_net_usage_table,df_net_usage_table.line_id == df_concat.line_id,how='left').drop(df_net_usage_table.line_id)

In [47]:
display(net_join_concat)

In [48]:
df_net_usage_features = net_join_concat.withColumn("sms_ave_id", func.col("sms_id") / func.col("tenure"))
df_net_usage_features = df_net_usage_features.withColumn("voice_duration_ave_id", func.col("voice_duration_id") / func.col("tenure"))
df_net_usage_features = df_net_usage_features.withColumn("voice_num_ave_id", func.col("voice_num_total_id") / func.col("tenure"))

In [49]:
# display(df_net_usage_features)

In [50]:
# df_net_usage_features.printSchema

In [51]:
df_net_usage_features.count()

phone_info

In [53]:
df_phone_info_old = df.select(func.col("customer_info.line_id").alias("line_id"),
                                 func.col("phone_info.phone_gen").alias("phone_gen"),
                               func.col("phone_info.operating_system").alias("operating_system"))

In [54]:
df_phone_info = df_phone_info_old.withColumn("update_col",
     func.when((func.col("operating_system") == "BYOP") | (func.col("operating_system") == "BYOD"), "BYOP(D)")
    .when((func.col("operating_system") == "IOS") | (func.col("operating_system") == "PROPRIETARY"), "IOS_PTY").otherwise(func.col("operating_system")))
   

In [55]:
display(df_phone_info)

In [56]:
df_phone_info.count()

support_history

In [58]:

df_support_history= df.select(func.col("customer_info.line_id").alias("line_id"),
                      func.explode("support_history").alias("support_history"))   

df_support_history=df_support_history.select("line_id","support_history.*")


In [59]:
df_support_case_num= df_support_history.groupby("line_id").count()
df_support_case_num=df_support_case_num.select(func.col("line_id").alias("line_id"),
                                              func.col("count").alias("case_num"))

In [60]:
display(df_support_case_num)

In [61]:
df_support_case_num.count()

join all the features together

In [63]:
#df_features= df_customer_info
customer_net = df_customer_info.join(df_net_usage_features,
                                     df_customer_info.line_id==df_net_usage_features.line_id,how='left').drop(df_net_usage_features.line_id)
customer_net_support = customer_net.join(df_support_case_num,
                                       df_support_case_num.line_id ==customer_net.line_id,how='left').drop(df_support_case_num.line_id)
customer_net_support_phone = df_phone_info.join(customer_net_support,
                                       df_phone_info.line_id ==customer_net_support.line_id,how='left').drop(customer_net_support.line_id)

customer_net_support_phone_status = customer_net_support_phone.join(df_status,customer_net_support_phone.line_id==df_status.line_id,how='left').drop(df_status.line_id)

# display(customer_net_support_phone_status)


In [64]:
display(customer_net_support_phone_status)

select final features

In [66]:
final_df = customer_net_support_phone_status.select(func.col("update_col").alias("operating_system_new"),
                                 func.col("phone_gen"),
                                 func.col("case_num"),
                                 func.col("lifetime_redemptions").alias("lifetime_redemptions"),
                                 func.col("lifetime_revenues").alias("lifetime_revenues"),     
                                 func.col("tenure"),
                                 func.col("sms_ave_id"),
                                 func.col("voice_duration_ave_id"),
                                 func.col("voice_num_ave_id"),
                                 func.col("status"))

In [67]:
# drop deactivation.isNull()
final_df_ = final_df.filter(final_df['tenure'].isNotNull())

In [68]:
display(final_df_)

In [69]:
final_df_.count()

In [70]:
# from pyspark.sql.functions import isnan

In [71]:
final_df_.cache()

In [72]:
final_df_.select("case_num").filter(final_df["case_num"].isNull()).count()

In [73]:
final_df_.select("lifetime_redemptions").filter(final_df["lifetime_redemptions"].isNull()).count()

In [74]:
final_df_.select("redem_revenues").filter(final_df["redem_revenues"].isNull()).count()

In [75]:
final_df_.select("sms_ave_id").filter(final_df["sms_ave_id"].isNull()).count()

In [76]:
final_df_.select("voice_duration_ave_id").filter(final_df["voice_duration_ave_id"].isNull()).count()
# final_df.select("voice_duration_ave_id").filter((isnan(final_df["voice_duration_ave_id"]))).count()

In [77]:
final_df_.select("voice_num_ave_id").filter(final_df["voice_num_ave_id"].isNull()).count()
# final_df.select("voice_num_ave_id").filter((isnan(final_df["voice_num_ave_id"]))).count()

In [78]:
final_df_.select("tenure").filter(final_df["tenure"].isNull()).count()
# final_df.select("tenure").filter((isnan(final_df["tenure"]))).count()

In [79]:
final_df_.select("operating_system_new").filter(final_df["operating_system_new"].isNull()).count()
# final_df.select("operating_system_new").filter((isnan(final_df["operating_system_new"]))).count()

In [80]:
final_df_.select("phone_gen").filter(final_df["phone_gen"].isNull()).count()